In [63]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 데이터 로드
data = pd.read_csv('20171104_test.csv', index_col=0)

# 형태소 분석기 초기화
okt = Okt()

# 기사의 내용을 형태소로 분석
data['tokenized_content'] = data['content'].apply(lambda x: ' '.join(okt.morphs(x)))

# TF-IDF 변환기를 초기화
tfidf_vectorizer = TfidfVectorizer()

# 형태소 분석된 내용을 벡터로 변환
tfidf_matrix = tfidf_vectorizer.fit_transform(data['tokenized_content'])

# 코사인 유사도 행렬 계산
cosine_sim = cosine_similarity(tfidf_matrix)

# 행렬의 통계 값 계산
# Compute statistics for the matrix
min_value = round(np.min(cosine_sim), 4)
max_value = round(np.max(cosine_sim), 4)
mean_value = round(np.mean(cosine_sim), 4)
median_value = round(np.median(cosine_sim), 4)
std_value = round(np.std(cosine_sim), 4)

min_value, max_value, mean_value, median_value, std_value

(0.0127, 1.0, 0.2894, 0.1647, 0.3401)

In [64]:
len(data)

10

In [ ]:
# 군집화
clusters = []
visited = np.zeros(cosine_sim.shape[0], dtype=bool)
for i in range(cosine_sim.shape[0]):
    if not visited[i]:
        similar_articles = np.where(cosine_sim[i] > 0.6)[0]
        clusters.append(similar_articles)
        visited[similar_articles] = True

# 각 군집에서 대표 기사만 남기기
representative_indices = [cluster[0] for cluster in clusters if len(cluster) > 0]

# 대표 기사만 포함된 데이터프레임 생성
filtered_data = data.iloc[representative_indices].reset_index(drop=True)
print(len(filtered_data))
filtered_data

6


,time,title,content,press,link,UTC-time,UTC-date,tokenized_content
0,2017-11-04 09:56:45,잠잠한 북한…트럼프 순방기간 도발 가능성은,\r\n\t\t\t잠잠한 북한…트럼프 순방기간 도발 가능성은[앵커] 북한은 최근 5...,연합뉴스TV언론사 선정,https://n.news.naver.com/mnews/article/422/000...,2017-11-04 00:56:45,2017-11-04,\r\n\t\t\t 잠잠한 북한 … 트럼프 순방 기간 도발 가능성 은 [ 앵커 ] ...
1,2017-11-04 09:01:08,CME 비트코인 거래 개시 소식에 계좌 개설 열풍,\n\n\n\n\n[아시아경제 황준호 기자] 세계 최대 파생상품 거래소인 미국 시...,아시아경제,https://n.news.naver.com/mnews/article/277/000...,2017-11-04 00:01:08,2017-11-04,\n\n\n\n\n [ 아시아 경제 황준호 기자 ] 세계 최대 파생상품 거래소 인 ...
2,2017-11-04 15:37:27,"스파크랩, 1천억 규모 블록체인 투자 펀드 출범",\n\t\t\t 스파크랩 그룹은 블록체인과 암호화 화폐 기업 투자 펀드인 ‘스파크체...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,2017-11-04 06:37:27,2017-11-04,\n\t\t\t 스파크 랩 그룹 은 블록 체인 과 암호 화 화폐 기업 투자 펀드 ...
3,2017-11-04 19:55:00,"""현직 경찰관에 배우 동원까지""…가상화폐 범죄 기승",\r\n\t\t\t【 앵커멘트 】 하루 거래액이 2조 원을 넘어서면서 비트코인 등...,MBN,https://n.news.naver.com/mnews/article/057/000...,2017-11-04 10:55:00,2017-11-04,\r\n\t\t\t 【 앵커 멘트 】 하루 거래 액 이 2조 원 을 넘어서면서 비트...
4,2017-11-05 01:34:20,"가상화폐, 금융시장의 새로운 쓰나미",\r\n비트코인·이더리움 등 1000종류상위 10개 시가총액만 150조원빠르고 거래...,T19면 TOP,https://n.news.naver.com/mnews/article/353/000...,2017-11-04 16:34:20,2017-11-04,\r\n 비트코인 · 이 더 리움 등 1000 종류 상위 10 개 시가총액 만 15...
5,2017-11-05 06:00:00,[박수찬의 軍]대통령 공약과 따로 노는 국방부의 北 미사일 대책,\r\n\r\n\r\n\r\n\r\n천궁 지대공미사일이 2일 충남 보령 대천사격장에...,세계일보언론사 선정,https://n.news.naver.com/mnews/article/022/000...,2017-11-04 21:00:00,2017-11-04,\r\n\r\n\r\n\r\n\r\n 천궁 지 대 공 미사일 이 2일 충남 보령 대...


In [67]:
filtered_data.to_csv('filtered_20171104_01.csv', encoding='utf-8-sig', index=False)

In [68]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np

# 데이터 로드
data = pd.read_csv('20171104_test.csv')

# sBERT 모델 로드
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# 각 기사의 내용을 임베딩
embeddings = model.encode(data['content'].tolist(), convert_to_tensor=True)

# 코사인 유사도 계산 함수
def cosine_similarity(embedding1, embedding2):
    return util.pytorch_cos_sim(embedding1, embedding2)

# 유사도 행렬 초기화
similarity_matrix = []

for i in range(len(data)):
    row = []
    for j in range(len(data)):
        similarity = cosine_similarity(embeddings[i], embeddings[j]).item()
        row.append(similarity)
    similarity_matrix.append(row)

# 행렬의 통계 값 계산
min_value = round(np.min(similarity_matrix), 4)
max_value = round(np.max(similarity_matrix), 4)
mean_value = round(np.mean(similarity_matrix), 4)
median_value = round(np.median(similarity_matrix), 4)
std_value = round(np.std(similarity_matrix), 4)

min_value, max_value, mean_value, median_value, std_value

(0.3614, 1.0, 0.6214, 0.5544, 0.2093)

In [69]:
# 군집화
clusters = []
visited = np.zeros(len(data), dtype=bool)
for i in range(len(data)):
    if not visited[i]:
        similar_articles = np.where(np.array(similarity_matrix[i]) > 0.6)[0]
        clusters.append(similar_articles)
        visited[similar_articles] = True

# 각 군집에서 대표 기사만 남기기
representative_indices = [cluster[0] for cluster in clusters if len(cluster) > 0]

# 대표 기사만 포함된 데이터프레임 생성
filtered_data = data.iloc[representative_indices].reset_index(drop=True)
print(len(filtered_data))
filtered_data

4


,date,time,title,content,press,link,UTC-time,UTC-date
0,2017-11-04,2017-11-04 09:56:45,잠잠한 북한…트럼프 순방기간 도발 가능성은,\r\n\t\t\t잠잠한 북한…트럼프 순방기간 도발 가능성은[앵커] 북한은 최근 5...,연합뉴스TV언론사 선정,https://n.news.naver.com/mnews/article/422/000...,2017-11-04 00:56:45,2017-11-04
1,2017-11-04,2017-11-04 09:01:08,CME 비트코인 거래 개시 소식에 계좌 개설 열풍,\n\n\n\n\n[아시아경제 황준호 기자] 세계 최대 파생상품 거래소인 미국 시...,아시아경제,https://n.news.naver.com/mnews/article/277/000...,2017-11-04 00:01:08,2017-11-04
2,2017-11-04,2017-11-04 15:37:27,"스파크랩, 1천억 규모 블록체인 투자 펀드 출범",\n\t\t\t 스파크랩 그룹은 블록체인과 암호화 화폐 기업 투자 펀드인 ‘스파크체...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,2017-11-04 06:37:27,2017-11-04
3,2017-11-05,2017-11-05 01:34:20,"가상화폐, 금융시장의 새로운 쓰나미",\r\n비트코인·이더리움 등 1000종류상위 10개 시가총액만 150조원빠르고 거래...,T19면 TOP,https://n.news.naver.com/mnews/article/353/000...,2017-11-04 16:34:20,2017-11-04


In [70]:
filtered_data.to_csv('filtered_20171104_02.csv', encoding='utf-8-sig', index=False)